In [74]:
from sklearn import tree

In [75]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [76]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import accuracy_score, classification_report

In [77]:
# perform basic data cleaning
data = pd.read_csv('machine_learning.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               50 non-null     int64  
 1   Unnamed: 0.1             50 non-null     int64  
 2   state                    50 non-null     object 
 3   mask_willingness         50 non-null     int64  
 4   vaccine_unwilling        50 non-null     int64  
 5   supply_percent_used      50 non-null     float64
 6   % population passed      50 non-null     float64
 7   election_result          50 non-null     int64  
 8   postvax_deaths           50 non-null     int64  
 9   prevax_deaths            50 non-null     int64  
 10  % population vaccinated  50 non-null     float64
dtypes: float64(3), int64(7), object(1)
memory usage: 4.4+ KB


In [78]:
# split for train and test data
X = data.drop(['state','election_result'],axis=1)
y = data['election_result'] 

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [79]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(37, 9)
(13, 9)
(37,)
(13,)


In [80]:
#create model
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(solver='lbfgs', random_state=1)
logreg

LogisticRegression(random_state=1)

In [81]:
# Train the data
logreg.fit(X_train, y_train)

/Users/justinobrien/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(random_state=1)

In [82]:
# Predict outcomes for test data set
predictions = logreg.predict(X_test)
pd.DataFrame({"Prediction": predictions, "Actual": y_test})

,Prediction,Actual
36,1,1
19,1,1
24,0,0
2,1,1
25,0,0
28,1,1
49,0,0
7,1,1
17,0,0
13,1,0


In [83]:
#validate the test
from sklearn.metrics import accuracy_score
accuracy_score(y_test, predictions)

0.9230769230769231

In [86]:
# Print the classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.83      1.00      0.91      0.91      0.82         6
          1       0.88      1.00      0.83      0.93      0.91      0.85         7

avg / total       0.93      0.92      0.91      0.92      0.91      0.83        13



In [87]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
#create instance
forrest = BalancedRandomForestClassifier(n_estimators=100, random_state=1)

forrest.fit(X_train, y_train)

BalancedRandomForestClassifier(random_state=1)

In [88]:
# Calculated the balanced accuracy score
y_pred = forrest.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.8571428571428572

In [89]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.75      1.00      0.71      0.86      0.85      0.73         6
          1       1.00      0.71      1.00      0.83      0.85      0.69         7

avg / total       0.88      0.85      0.87      0.84      0.85      0.71        13



In [98]:
#method 3 decision tree
# Creating StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [99]:
# Fitting Standard Scaller
X_scaler = scaler.fit(X_train)
# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [100]:

from sklearn.tree import DecisionTreeClassifier
# Creating the decision tree classifier instance.
tree = tree.DecisionTreeClassifier()
# Fitting the model.
tree = tree.fit(X_train_scaled, y_train)

AttributeError: 'DecisionTreeClassifier' object has no attribute 'DecisionTreeClassifier'

In [93]:
# Making predictions using the testing data
predictions = tree.predict(X_test)
predictions

array([1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0])

In [94]:
# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)
acc_score

0.8461538461538461

In [95]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.75      1.00      0.86         6
           1       1.00      0.71      0.83         7

    accuracy                           0.85        13
   macro avg       0.88      0.86      0.85        13
weighted avg       0.88      0.85      0.84        13

